In [22]:
import torch
import numpy as np
from scipy import signal
import pickle
import timeit

In [23]:
# 读取数据
with open('data\\180007_1695193424_2023-09-20_15-03-44.pkl', 'rb') as f:
    data = pickle.load(f)
    
print(data.keys())

dict_keys(['data_list', 'gps_lock_flag', 'real_lat', 'real_lon', 'real_alt'])


In [24]:
complex_data = data['data_list'][0]["recv_data"]
print(complex_data.dtype)
print(complex_data.shape[0])
complex_data = complex_data[20000:520000]

complex64
5000000


In [25]:
# # numpy corr
# start_time = timeit.default_timer()

# np_correlate = np.correlate(complex_data, complex_data, mode='full')

# end_time = timeit.default_timer()
# duration = end_time - start_time
# print(f"运行时长: {duration} 秒")

In [26]:
# scipy corr
start_time = timeit.default_timer()

scipy_correlate = signal.correlate(complex_data, complex_data, mode='full')

end_time = timeit.default_timer()
duration = end_time - start_time
print(f"运行时长: {duration} 秒")

# 调用了INTEL MKL库

运行时长: 0.05523659999926167 秒


In [27]:
# torch corr cpu
M = complex_data.shape[0]
N = complex_data.shape[0]

start_time = timeit.default_timer()


x = torch.from_numpy(complex_data)
print(x.dtype)

cpu_torch_correlate = torch.fft.ifft((torch.fft.fft(x, n=M+N-1))*torch.conj(torch.fft.fft(x, n=M+N-1)))

end_time = timeit.default_timer()
duration = end_time - start_time
print(f"运行时长: {duration} 秒")

torch.complex64
运行时长: 0.031870700000581564 秒


In [28]:
print(torch.cuda.is_available())

M = complex_data.shape[0]
N = complex_data.shape[0]

start_time = timeit.default_timer()

x = torch.from_numpy(complex_data)
y = x.cuda()

print(y.dtype)

cpu_torch_correlate = torch.fft.ifft((torch.fft.fft(y, n=M+N-1))*torch.conj(torch.fft.fft(y, n=M+N-1)))

end_time = timeit.default_timer()
duration = end_time - start_time
print(f"运行时长: {duration} 秒")

# 50w个点快了一些

True
torch.complex64
运行时长: 0.006664900000032503 秒
